In [3]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import csv
from peft import PeftModel
from bs4 import BeautifulSoup




/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# 控制是否挂载 LoRA
use_lora = True  # 改为 False 即不加载 LoRA
model_name = "/root/Qwen2.5-7B-Instruct"
lora_adapter_path = "/root/saves/Qwen2.5-7B-Instruct/lora/train_2025-04-27-07-02-49/"

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 加载基础模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

# 根据控制变量加载 LoRA
if use_lora:
    model = PeftModel.from_pretrained(model, lora_adapter_path)
    print("loaded lora succeed")
else:
    print("loaded lora failed")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ 已加载LoRA适配器。


In [3]:
# # load model from local
# model_name = "/root/Qwen2.5-7B-Instruct"
# lora_adapter_path = "/root/saves/Qwen2.5-7B-Instruct/lora/train_2025-04-27-07-02-49/adapter_model.safetensors"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def clean_html(raw_html):
    if pd.isna(raw_html):
        return ""
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text(separator=" ", strip=True)

In [4]:
train_df = pd.read_csv("/root/autodl-tmp/data/salary_labelled_development_set.csv")  

def row_to_text(row, include_label=True):
    job_title = row["job_title"]
    job_details = clean_html(row["job_ad_details"])
    # job_details = row["job_ad_details"]
    location = row["nation_short_desc"]

    base_text = f"Job Title: {job_title}\nJob Description: {job_details}\nLocation: {location}"
    if include_label:
        base_text += f"\ny_true: {row['y_true']}"
    return base_text

In [ ]:
train_df = pd.read_csv("/root/autodl-tmp/data/salary_labelled_development_set.csv")  

def row_to_text(row, include_label=True):
    job_title = row["job_title"]
    job_details = clean_html(row["job_ad_details"])
    # job_details = row["job_ad_details"]
    location = row["nation_short_desc"]

    base_text = f"Job Title: {job_title}\nJob Description: {job_details}\nLocation: {location}"
    if include_label:
        base_text += f"\ny_true: {row['y_true']}"
    return base_text

# use first 10 data in train set as few-shot
shots = "\n\n".join([f"example{i+1}：\n{row_to_text(train_df.iloc[i])}" for i in range(10)])

In [8]:
# use first 10 data in train set as few-shot
shots = "\n\n".join([f"example{i+1}：\n{row_to_text(train_df.iloc[i])}" for i in range(10)])

In [10]:
# 指定保存路径
filename = "shots_examples.txt"

# 保存到文件
with open(filename, "w", encoding="utf-8") as f:
    f.write(shots)

In [9]:
shots

"example1：\nJob Title: Financial Account - Call Center Agent - Up to 34k\nJob Description: Job Opening Financial Account - Call Center Agent - Up to 34k Job Industry Telecommunications Job Type Full-Time Experience Level Entry Level Date Posted 2022-10-27 Job Location Pasig BlvdPasig1000NCRPhilippines Company Information Sapient\n \n Pasig Blvd \n Cebu, Central Visayas \n 6019 \n Sapient is Philippine-based BPO that provides a range of outsourcing services from consulting services, IT-enabled services, and call center services primarily catering small and medium based enterprises. Job Description Job Responsibilities: Answers phone calls and provides important information/ assistance to clients Checks mail, fax and internet mail to provide customer assistance Communicates with customer on the phone or using written correspondence to take care of concerns Answer participant questions, , as well as talk to participants to achieve full understanding of what critical information are being 

In [7]:
test_df = pd.read_csv("/root/autodl-tmp/data/salary_labelled_test_set.csv")


test_df["predicted"] = ""
test_df["true_label"] = test_df["y_true"].astype(str)
results = []
correct=0

for i, row in test_df.iterrows():
    query = f"query：\n{row_to_text(row, include_label=False)}\nyour_predict：y_true ="

    user_prompt = (
        "You are a classification assistant. Below are some job descriptions along with their corresponding salaries (y_true)：\n\n"
        "Please only give me answer, no more other things.The stuctrue shold be like 100-200-AUS-MONTHLY.(No commas are required between numbers)"
        "If you think there is no mention of salary, please give me 0-0-None-None."
        f"{shots}\n\n{query}"
        # f"{query}"
    )

    messages = [
        # {"role": "system", "content": ""},
        {"role": "user", "content": user_prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    outputs = model.generate(
        **model_inputs,
        max_new_tokens=30,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    gen_ids = outputs[0][model_inputs["input_ids"].shape[-1]:]
    prediction = tokenizer.decode(gen_ids, skip_special_tokens=True).strip().split("\n")[0]

    results.append({
        "index": i,
        "predicted": prediction,
        "true_label": str(row["y_true"])
    })
    if prediction == str(row["y_true"]):
        correct+=1
    # save
    if (i + 1) % 5 == 0:
        checkpoint_df = pd.DataFrame(results)
        filename = f"qwen7/qwen-7b-salary_predictions2.csv"
        checkpoint_df.to_csv(filename, index=False)
        print(f"{i+1},save result to：{filename}")
    number=i+1
    # if i ==220:
    #     break


filename = f"qwen7/qwen-7b-salary_predictions2.csv"
final_df = pd.DataFrame(results)
final_df.to_csv(filename, index=False)

5,save result to：qwen7/qwen-7b-salary_predictions2.csv
10,save result to：qwen7/qwen-7b-salary_predictions2.csv
15,save result to：qwen7/qwen-7b-salary_predictions2.csv
20,save result to：qwen7/qwen-7b-salary_predictions2.csv
25,save result to：qwen7/qwen-7b-salary_predictions2.csv
30,save result to：qwen7/qwen-7b-salary_predictions2.csv
35,save result to：qwen7/qwen-7b-salary_predictions2.csv
40,save result to：qwen7/qwen-7b-salary_predictions2.csv
45,save result to：qwen7/qwen-7b-salary_predictions2.csv
50,save result to：qwen7/qwen-7b-salary_predictions2.csv
55,save result to：qwen7/qwen-7b-salary_predictions2.csv
60,save result to：qwen7/qwen-7b-salary_predictions2.csv
65,save result to：qwen7/qwen-7b-salary_predictions2.csv
70,save result to：qwen7/qwen-7b-salary_predictions2.csv
75,save result to：qwen7/qwen-7b-salary_predictions2.csv
80,save result to：qwen7/qwen-7b-salary_predictions2.csv
85,save result to：qwen7/qwen-7b-salary_predictions2.csv
90,save result to：qwen7/qwen-7b-salary_prediction

In [8]:

correct

457

In [9]:
number

567

In [10]:
accuracy=correct/number


In [ ]:
print(f"accuracy：{accuracy:.2%}")
print(f"save result to：{filename}")

accuracy：80.60%
save result to：qwen7/qwen-7b-salary_predictions2.csv


: 